In [1]:
import numpy as np
import pandas as pd 
import os

home_dir = os.environ.get('HOME')
directory_path = os.path.join(home_dir, 'Data', 'protein', 'cafa-5-protein-function-prediction')

for dirname, _, filenames in os.walk(directory_path):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/Users/zane/Data/protein/cafa-5-protein-function-prediction/sample_submission.tsv
/Users/zane/Data/protein/cafa-5-protein-function-prediction/.DS_Store
/Users/zane/Data/protein/cafa-5-protein-function-prediction/IA.txt
/Users/zane/Data/protein/cafa-5-protein-function-prediction/Test (Targets)/.DS_Store
/Users/zane/Data/protein/cafa-5-protein-function-prediction/Test (Targets)/testsuperset.fasta
/Users/zane/Data/protein/cafa-5-protein-function-prediction/Test (Targets)/testsuperset-taxon-list.tsv
/Users/zane/Data/protein/cafa-5-protein-function-prediction/Train/train_taxonomy.tsv
/Users/zane/Data/protein/cafa-5-protein-function-prediction/Train/.DS_Store
/Users/zane/Data/protein/cafa-5-protein-function-prediction/Train/train_terms.tsv
/Users/zane/Data/protein/cafa-5-protein-function-prediction/Train/go-basic.obo
/Users/zane/Data/protein/cafa-5-protein-function-prediction/Train/train_sequences.fasta


- train_sequences.fasta - 训练集中蛋白质的氨基酸序列
- train_terms.tsv - 蛋白质的训练集和相应的带注释的 GO 术语
- train_taxonomy.tsv - 训练集中蛋白质的分类ID
- go-basic.obo - 本体图结构
- testsuperset.fasta - 应进行预测的蛋白质的氨基酸序列
- testsuperset-taxon-list.tsv - 测试超集中蛋白质的分类ID（注意：您可能需要使用 encoding="ISO-8859-1" 来读取 pandas 中的此文件）
- IA.txt - 每个术语的信息积累。这用于衡量精确度和召回率（参见评估）
- Sample_submission.csv - 正确格式的示例提交文件

In [2]:
from Bio import SeqIO

sequences = []
for record in SeqIO.parse(os.path.join(directory_path, 'Train', 'train_sequences.fasta'), "fasta"):
    sequences.append(str(record.seq))

df = pd.DataFrame(sequences, columns=['sequence'])

In [3]:
df

,sequence
0,MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIP...
1,MTEYRNFLLLFITSLSVIYPCTGISWLGLTINGSSVGWNQTHHCKL...
2,MRLSSSPPRGPQQLSSFGSVDWLSQSSCSGPTHTPRPADFSLGSLP...
3,MGGEAGADGPRGRVKSLGLVFEDESKGCYSSGETVAGHVLLEAAEP...
4,MVETNSPPAGYTLKRSPSDLGEQQQPPRQISRSPGNTAAYHLTTAM...
...,...
142241,METEVDDFPGKASIFSQVNPLYSNNMKLCEAERYDFQHSEPKTMKS...
142242,MSAAASAEMIETPPVLNFEEIDYKEIEVEEVVGRGAFGVVCKAKWR...
142243,MADKGPILTSVIIFYLSIGAAIFQILEEPNLNSAVDDYKNKTNNLL...
142244,MGRKKIQITRIMDERNRQVTFTKRKFGLMKKAYELSVLCDCEIALI...
